
## Analysis of access to electricity (source: the World Bank)

The World Bank offers a large amount of open data. In this exercise we will focus on the percentage of population having access to electricity in all the countries of the world from 1960 to 2014.  The data can be obtained from this link: https://data.worldbank.org/indicator/EG.ELC.ACCS.ZS. After selecting the CSV format you will get a zip file containing, among others, these two files:

API_EG.ELC.ACCS.ZS_DS2_en_csv_v2.csv
Metadata_Country_API_EG.ELC.ACCS.ZS_DS2_en_csv_v2.csv
After taking a look at the content of these files, write a program named electricityAnalysis.py/java which, given a range of years (e.g., 1970 and 1980), prints on the screen the average number of countries of each of the regions (South Asia, North America, etc.) having at least 99% of electricity for all the years between the indicated range. The output must be ordered in descendent order by the percentage value, and it should look like this example (note: these data are invented): 




In [160]:
# Import requiered libriaries

from pyspark.sql import SparkSession
from pyspark.sql.types import *
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [161]:
# Set spark session
spark_session = SparkSession \
    .builder \
    .getOrCreate()

In [164]:
# Create struct type to define theirs type

struct_ds = [StructField('CountryName', StringType(),False), \
                       StructField('CountryCode', StringType(), False), \
                       StructField('IndicatorName', StringType(), False), \
                       StructField('IndicatorCode', StringType(), False)]


In [165]:
for years in range(1960,2018):
    struct_ds.append(StructField("_" +str(years), DoubleType(),False))
    
schema = StructType(struct_ds)

In [166]:
# Load dataset

dataset = spark_session.read.csv(
        path="/home/master/Descargas/API/API_2.csv",
        sep=",",
        header=True,
        schema=schema)

In [167]:
dataset.show(25)

+--------------------+------------+--------------------+--------------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------+----------------+----------------+----------------+
|         CountryName| CountryCode|       IndicatorName| IndicatorCode| _1960| _1961| _1962| _1963| _1964| _1965| _1966| _1967| _1968| _1969| _1970| _1971| _1972| _1973| _1974| _1975| _1976| _1977| _1978| _1979| _1980| _1981| _1982| _1983| _1984| _1985| 

In [168]:
# Load second dataset

dataset2 = spark_session.read.csv(
        path="/home/master/Descargas/API/Metadata_Country.csv",
        sep=",",
        header=True)

In [169]:
dataset2.show(2)

+------------+--------------------+-----------+------------+-----------+----+
|Country Code|              Region|IncomeGroup|SpecialNotes|  TableName| _c5|
+------------+--------------------+-----------+------------+-----------+----+
|         ABW|Latin America & C...|High income|        null|      Aruba|null|
|         AFG|          South Asia| Low income|        null|Afghanistan|null|
+------------+--------------------+-----------+------------+-----------+----+
only showing top 2 rows



In [170]:
# Join two datasets on their common column.

dataset_complete = dataset.join(dataset2,dataset["CountryCode"] == dataset2["Country Code"])
        

In [171]:
display((dataset_complete).toPandas())

,CountryName,CountryCode,IndicatorName,IndicatorCode,_1960,_1961,_1962,_1963,_1964,_1965,...,_2014,_2015,_2016,_2017,Country Code,Region,IncomeGroup,SpecialNotes,TableName,_c5
0,Aruba,ABW,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,100.000000,100.000000,100.000000,100.000000,ABW,Latin America & Caribbean,High income,None,Aruba,None
1,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,89.500000,71.500000,97.700000,97.700000,AFG,South Asia,Low income,None,Afghanistan,None
2,Angola,AGO,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,32.000000,42.000000,40.624527,41.886230,AGO,Sub-Saharan Africa,Lower middle income,None,Angola,None
3,Albania,ALB,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,100.000000,100.000000,100.000000,100.000000,ALB,Europe & Central Asia,Upper middle income,None,Albania,None
4,Andorra,AND,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,100.000000,100.000000,100.000000,100.000000,AND,Europe & Central Asia,High income,None,Andorra,None
5,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,87.342739,89.130121,89.678685,90.273687,ARB,None,None,Arab World aggregate. Arab World is composed o...,Arab World,None
6,United Arab Emirates,ARE,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,100.000000,100.000000,100.000000,100.000000,ARE,Middle East & North Africa,High income,None,United Arab Emirates,None
7,Argentina,ARG,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,100.000000,99.823578,99.966881,100.000000,ARG,Latin America & Caribbean,Upper middle income,None,Argentina,None
8,Armenia,ARM,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,100.000000,100.000000,100.000000,100.000000,ARM,Europe & Central Asia,Upper middle income,None,Armenia,None
9,American Samoa,ASM,Access to electricity (% of population),EG.ELC.ACCS.ZS,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,ASM,East Asia & Pacific,Upper middle income,None,American Samoa,None


In [173]:
# Check the types are correct
dataset_complete.printSchema()

root
 |-- CountryName: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- IndicatorName: string (nullable = true)
 |-- IndicatorCode: string (nullable = true)
 |-- _1960: double (nullable = true)
 |-- _1961: double (nullable = true)
 |-- _1962: double (nullable = true)
 |-- _1963: double (nullable = true)
 |-- _1964: double (nullable = true)
 |-- _1965: double (nullable = true)
 |-- _1966: double (nullable = true)
 |-- _1967: double (nullable = true)
 |-- _1968: double (nullable = true)
 |-- _1969: double (nullable = true)
 |-- _1970: double (nullable = true)
 |-- _1971: double (nullable = true)
 |-- _1972: double (nullable = true)
 |-- _1973: double (nullable = true)
 |-- _1974: double (nullable = true)
 |-- _1975: double (nullable = true)
 |-- _1976: double (nullable = true)
 |-- _1977: double (nullable = true)
 |-- _1978: double (nullable = true)
 |-- _1979: double (nullable = true)
 |-- _1980: double (nullable = true)
 |-- _1981: double (nullable = true)
 |-- 

In [189]:
# Take input from keyboard for year range

start_year = int(input("Write a start year "))
end_year = int(input("Write a end year "))

Write a start year 1999
Write a end year 2007


In [190]:
years=[]

In [191]:
# Create years for sql query
for year in range(start_year,end_year): 
    years.append("_{} >= 99 AND ".format(year))

In [192]:
# Make years string
year_formatted=(str(years)[1:-1].replace("'"," " )).replace(",","" )

In [193]:
year_formatted

' _1999 >= 99 AND    _2000 >= 99 AND    _2001 >= 99 AND    _2002 >= 99 AND    _2003 >= 99 AND    _2004 >= 99 AND    _2005 >= 99 AND    _2006 >= 99 AND  '

In [194]:
# Make queries for two cases, which are regions' countries whose population have an access to electricity 
# more than 99%  and number of countries per region in order to calculate % of electricity access

dataset_complete.createOrReplaceTempView("table2")

df_99  = spark_session.sql("SELECT Region,Count(CountryName) as number from table2 where "+ year_formatted + " Region IS NOT NULL GROUP BY Region" ).toPandas()

df = spark_session.sql("SELECT Region,Count(CountryName) as number from table2 where Region IS NOT NULL GROUP BY Region" ).toPandas()



In [195]:
df_99

,Region,number
0,Europe & Central Asia,48
1,North America,3
2,East Asia & Pacific,12
3,Middle East & North Africa,5
4,Latin America & Caribbean,3


In [196]:
df

,Region,number
0,South Asia,8
1,Sub-Saharan Africa,48
2,Europe & Central Asia,58
3,North America,3
4,East Asia & Pacific,37
5,Middle East & North Africa,21
6,Latin America & Caribbean,42


In [197]:
# Merge two dataframe

df_merged =pd.merge(df_99, df, left_on='Region', right_on='Region', how='left')

In [201]:
df_merged

,Region,number_x,number_y,Percentage
0,Europe & Central Asia,48,58,83.0
1,North America,3,3,100.0
2,East Asia & Pacific,12,37,32.0
3,Middle East & North Africa,5,21,24.0
4,Latin America & Caribbean,3,42,7.0


In [203]:
# Calculate percentage and prepare format

df_merged["Percentage"] = round(df_merged["number_x"]/df_merged["number_y"].div(100),0)
df_end = df_merged[["Region","Percentage"]].sort_values(by=["Percentage"], ascending=False)
df_end["Percentage"] = df_end["Percentage"].astype(str) + '%'
df_end

,Region,Percentage
1,North America,100.0%
0,Europe & Central Asia,83.0%
2,East Asia & Pacific,32.0%
3,Middle East & North Africa,24.0%
4,Latin America & Caribbean,7.0%
